<a href="https://colab.research.google.com/github/georgeliu1998/keras_model_tuning/blob/master/keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import sklearn

from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.datasets import load_iris, load_digits, load_boston

import pickle
from time import time
import xgboost as xgb

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical

In [13]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6828979688374410983
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14382776406876984864
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7828053239943898867
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11281553818
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6227211098185073780
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [14]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
seed = np.random.RandomState(6)

In [0]:
iris = datasets.load_iris()

X = iris['data']
y = iris['target']

In [23]:
clf = xgb.XGBClassifier()
cv = StratifiedKFold(n_splits=5, random_state=seed)

scores = cross_val_score(clf, X, y, cv=cv)

print("Mean Accuracy: {:.2%}, Standard Deviation: {:.2%}".format(scores.mean(), scores.std()))


Mean Accuracy: 95.33%, Standard Deviation: 3.40%


In [24]:
print("X shape: {}, y shape: {}".format(X.shape, y.shape))

X shape: (150, 4), y shape: (150,)


In [0]:
class SequentialModel:
  
  def __init__(self, num_layers, num_units, 
               activation, activation_out, 
               loss, initializer, optimizer, 
               metrics, epochs, batch_size):
    """
    Params:
      num_layers: int, number of layers of the model (excluding the input layer)
      num_units: tuple, number of units in each layer(excluding the input layer)
      activation: 
      activation_out:
      loss:
      initializer:
      optimizer:
      metrics:
      epochs:
      batch_size:
    """
    self.num_layers = num_layers
    self.num_units = num_units
    self.activation = activation
    self.activation_out = activation_out
    self.loss = loss
    self.initializer = initializer
    self.optimizer = optimizer
    self.metrics = metrics
    self.epochs = epochs
    self.batch_size = batch_size
    # Initialize the sequential model
    self.model = Sequential()
  
    
  def build_model(self):
    """
    Adds layers and compiles the model
    """
    # Loop thru all the layers
    for i in range(self.num_layers):
      # Different layers should have different setups
      if i == 0: # first layer
        self.model.add(Dense(units=self.num_units[i],
                             input_dim=X.shape[1],
                             #kernel_initializer=initializer,
                             activation=activation)) 
      elif i+1 == self.num_layers: # output layer
        self.model.add(Dense(units=self.num_units[i],
                             kernel_initializer=initializer,
                             activation=activation_out))
      else:
        self.model.add(Dense(units=self.num_units[i],
                            kernel_initializer=initializer,
                            activation=activation))
    
    self.model.compile(loss=self.loss,
                       optimizer=self.optimizer,
                       metrics=self.metrics)
      
  
  
  def evaluate_model(self, X, y, n_splits=5):
    """
    Evaluates the model using cross-validation.
    
    Params:
      X: np.array, features
      y: np.array, labels
      n_splits: int, number of folds for the cross-validation
    """
    score_lst = []
    t1 = time()
    
    print("Starting {}-fold cross-validation...".format(n_splits))
    
    kfold = StratifiedKFold(n_splits=n_splits, 
                            shuffle=True, 
                            random_state=seed)
    
    for train_index, test_index in kfold.split(X, y):
      # One-hot encoding
      y_cat = to_categorical(y)
      
      self.model.fit(X[train_index], 
                     y_cat[train_index], 
                     epochs=self.epochs,
                     batch_size=self.batch_size,
                     verbose=0)
      
      scores = self.model.evaluate(X[test_index], 
                                   y_cat[test_index], 
                                   verbose=0)
      # The second item is accuracy
      score_lst.append(scores[1])
      
    t2 = time()
    t = t2 - t1
    # Convert time to mintues
    t /= 60

    print("Finished cross-valiation. Took {:.1} mintues.".format(t))
      
    # Convert to np.array and calculate mean and sd
    score_lst = np.array(score_lst)
    mean_acc = score_lst.mean()
    sd_acc = score_lst.std()
    
    print("Mean Accuracy: {:.2%}, Standard Deviation: {:.2%}".format(mean_acc, sd_acc))
    
    return mean_acc

In [26]:
num_layers = 2
num_units = (4, 3) 
activation = 'relu'
activation_out = 'softmax'
loss = 'binary_crossentropy'
initializer = 'random_uniform'
optimizer = 'adam'
metrics = ['accuracy']
epochs = 100
batch_size = 5


model = SequentialModel(num_layers, num_units,
                        activation, activation_out, 
                        loss, initializer, optimizer, 
                        metrics, epochs, batch_size)

model.build_model()
model.evaluate_model(X, y)

Starting 5-fold cross-validation...

Finished cross-valiation. Took 0.9 mintues.
Mean Accuracy: 98.22%, Standard Deviation: 1.66%


0.9822222352027893

In [46]:
print("Starting {}fold cross-validation...\n".format(10))

Starting 10fold cross-validation...



## TO-DO's

1. OOB
2. 